In [247]:
import pandas as pd

airports = pd.read_csv('data/airports.csv')

In [248]:
airportLocations = {}
for index, row in airports.iterrows():
    airportLocations[row["IATA_CODE"]] = (row["LATITUDE"], row["LONGITUDE"])

In [374]:
import cPickle
import datetime
import matplotlib.pyplot as plt
import mplleaflet

origin = 'JFK'
destination = 'ATL'
direct = False

if direct:
    policyFile = 'Results/Winter/Direct/' + origin + 'to' + destination + '_direct_policy.pkl'
    valueFile = 'Results/Winter/Direct/' + origin + 'to' + destination + '_direct_value.pkl'
else:
    policyFile = 'Results/Winter/Connected/' + origin + 'to' + destination + '_policy.pkl'
    valueFile = 'Results/Winter/Connected/' + origin + 'to' + destination + '_value.pkl'

optimal_policy = {}
with open(policyFile, "rb") as input_file:
    optimal_policy = cPickle.load(input_file)

value_function = {}
with open(valueFile, "rb") as input_file:
    value_function = cPickle.load(input_file)

state = (origin, datetime.datetime(2015, 1, 11, 8, 30), 0)
print 'Value at start state is ', value_function[state]
path = [(state, None)]
actions = []

startTime = optimal_policy[state][1]

while True:
    print '\n'
    print 's: ', state, value_function[state]
    if state[0] == destination:
        finalTime = state[1]
        break
    else:
        action = optimal_policy[state]
        print 'a: ', action
        actions.append(action[1])
        state = (action[2],action[3],state[2]+1)
        path.append((state,action,state[2]+1))

airportsVisited = [i[0][0] for i in path]
print '\nairports we visit: ', airportsVisited

flightsToTake = [(i[1][0],i[1][4]) for i in path[1:]]
print 'flights to take: ', flightsToTake

points = []
for airport in airportsVisited:
    points.append(airportLocations[airport])

lats = [i[0] for i in points]
lngs = [i[1] for i in points]

if direct:
    plt.plot(lngs, lats,'ob-')
else:
    plt.plot(lngs, lats,'or-')

# mplleaflet.show()
print 'hours it takes: ', (divmod((finalTime - startTime).total_seconds(), 60)[0])/60




Value at start state is  -466.948030579


s:  ('JFK', datetime.datetime(2015, 1, 11, 8, 30), 0) -466.948030579
a:  ('B6583', datetime.datetime(2015, 1, 11, 11, 17), 'MCO', datetime.datetime(2015, 1, 11, 17, 22), 165.0)


s:  ('MCO', datetime.datetime(2015, 1, 11, 17, 22), 1) -141.981285479
a:  ('WN1527', datetime.datetime(2015, 1, 11, 17, 57), 'ATL', datetime.datetime(2015, 1, 11, 20, 9), 80.0)


s:  ('ATL', datetime.datetime(2015, 1, 11, 20, 9), 2) 0

airports we visit:  ['JFK', 'MCO', 'ATL']
flights to take:  [('B6583', 165.0), ('WN1527', 80.0)]
hours it takes:  8.86666666667
